# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
def priest(sex, age, respRate, oxySat, heartRate, systolicBP, temp, alertness, inspiredOxy, perfStatus):
    """Return raw score as percentile
    
    >>> print(priest("MALE", 80, 11, .95, 50, 11, 36, "Confused or not alert", "Air", "BED/CHAIR BOUND, NO SELF-CARE"))
    0.59
    
    >>> print(priest("MALE", 81, 8, .9, 30, 220, 34, "Confused or not alert", "Supplemental Oxygen", "BED/CHAIR BOUND, NO SELF-CARE"))
    0.99
    
    >>> print(priest("MALE", 80, 11, .95, 50, 11, 36, "Confused or not alert", "Pure Oxygen", "BED/CHAIR BOUND, NO SELF-CARE"))
    Traceback (most recent call last):
        ...
    ValueError: Pure Oxygen is not a valid value for inspired oxygen.
    """
    points = 0
    
    if sex.upper() == "MALE":
        points += 1
    elif sex.upper() != "FEMALE":
        raise ValueError(str(sex) + " is not a valid value for sex.")
    
    if 16 <= int(age) <= 49:
        pass
    elif 50 <= int(age) <= 65:
        points += 2
    elif 66 <= int(age) <= 80:
        points += 3
    elif int(age) > 80:
        points += 4
    else: 
        raise ValueError(age + " is not a valid value for age.")
        
    if int(respRate) < 9:
        points += 3
    elif 9 <= int(respRate) <= 11:
        points += 1
    elif 12 <= int(respRate) <= 20:
        pass
    elif 21 <= int(respRate) <= 24:
        points += 2
    elif int(respRate) > 24:
        points += 3
    
        
    if 1 >= float(oxySat) > .95:
        pass
    elif .94 <= float(oxySat) <= .95:
        points += 1
    elif .92 <= float(oxySat) <= .93:
        points += 2
    elif 0 <= float(oxySat) < .92:
        points += 3
    else:
        raise ValueError(oxySat + " is not a valid value for oxygen saturation.")
        
    if int(heartRate) < 41:
        points += 3
    elif 41 <= int(heartRate) <= 50:
        points += 1
    elif 51 <= int(heartRate) <= 90:
        pass
    elif 91 <= int(heartRate) <= 110:
        points += 1
    elif 111 <= int(heartRate) <= 130:
        points += 2
    elif int(heartRate) > 130:
        points += 3
        
    if int(systolicBP) < 91:
        points += 3
    elif 91 <= int(systolicBP) <= 100:
        points += 2
    elif 101 <= int(systolicBP) <= 110:
        points += 1
    elif 111 <= int(systolicBP) <= 219:
        pass
    elif int(systolicBP) > 219:
        points += 3
        
    if float(temp) < 35.1:
        points += 3
    elif 35.1 <= float(temp) <= 36:
        points += 1
    elif 36.1 <= float(temp) <= 38:
        pass
    elif 38.1 <= float(temp) <= 39:
        points += 1
    elif float(temp) > 39:
        points += 2
        
    if alertness.upper() == "CONFUSED OR NOT ALERT":
        points += 3
    elif alertness.upper() != "ALERT":
        raise ValueError(str(alertness) + " is not a valid value for alertness.")
    
    if inspiredOxy.upper() == "SUPPLEMENTAL OXYGEN":
        points += 2
    elif inspiredOxy.upper() != "AIR":
        raise ValueError(str(inspiredOxy) + " is not a valid value for inspired oxygen.")
    
    if perfStatus.upper() == "LIMITED STRENUOUS ACTIVITY, CAN DO LIGHT ACTIVITY":
        points += 1
    elif perfStatus.upper() == "LIMITED ACTIVITY, CAN SELF-CARE":
        points += 2
    elif perfStatus.upper() == "LIMITED SELF-CARE":
        points += 3
    elif perfStatus.upper() == "BED/CHAIR BOUND, NO SELF-CARE":
        points += 4
    elif perfStatus.upper() != "UNRESTRICTED NORMAL ACTIVITY":
        raise ValueError(str(perfStatus) + " is not a valid value for performance status.")
    
    if 0 <= points <= 1:
        percentile = .01
    elif 2 <= points <= 3:
        percentile = .02
    elif points == 4:
        percentile = .03
    elif points == 5:
        percentile = .09
    elif points == 6:
        percentile = .15
    elif points == 7:
        percentile = .18
    elif points == 8:
        percentile = .22
    elif points == 9:
        percentile = .26
    elif points == 10:
        percentile = .29
    elif points == 11:
        percentile = .34
    elif points == 12:
        percentile = .38
    elif points == 13:
        percentile = .46
    elif points == 14:
        percentile = .47
    elif points == 15:
        percentile = .49
    elif points == 16:
        percentile = .55
    elif 17 <= points <= 25:
        percentile = .59
    elif points >= 26:
        percentile = .99
    
    return percentile
        

if __name__ == "__main__":
    import doctest
    doctest.testmod()
    

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [2]:
import requests


def find_hospital(age, sex, risk):
    """Return name of hospital where to send patient.
    
    >>> find_hospital(age=40, sex="male", risk=0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(age=81, sex="male", risk=0.1)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(age=40, sex="female", risk=0.1)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(age=40, sex="female", risk=0.3)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(age=40, sex="male", risk='')
    """
    requestParamsStr = "age=" + str(age) + "&sex=" + sex + "&risk_pct=" + str(risk)
    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?" + requestParamsStr
    r = requests.get(url)
    data = r.json()
    try:
        hospital = data['hospital']
        return hospital
    except KeyError:
        return None

    
if __name__ == "__main__":
    import doctest
    doctest.testmod()


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [3]:
from bs4 import BeautifulSoup


def get_address(searchHospital):
    """Return address of specific hospital in Atlanta, Georgia.
    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    >>> get_address('Select Specialty Hospital - Northeast Atlanta')
    '1821 CLIFTON ROAD NE'
    """
    
    url = "https://www.officialusa.com/stateguides/health/hospitals/georgia.html"
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36" }
    results =[]
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    tableContainer = soup.find_all('div', {"id": "hospitalscity"})[0]
    hospitalTable = tableContainer.find_all('table')[0]
    alreadyFound = False
    rows = hospitalTable.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        if len(columns) == 0:
            continue

        if columns[0].text == "ATLANTA" and columns[1].text == searchHospital.upper():
            address = columns[3].text
            
            if alreadyFound:
                raise KeyError

            alreadyFound = True
                
    
    if not alreadyFound:
        raise KeyError
    return address


if __name__ == "__main__":
    import doctest
    doctest.testmod()

                
 


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [4]:
import csv

def process_people(filename):
    """Returns dictionary of each processed patient. 
    The dict keys are the patient numbers.
    The dict values are lists containing the other items in a row, 
    as well as the hospital name and address.
    """
    
    processedPeople = {}
    
    with open(filename, newline='') as f:
        reader = csv.DictReader(f, delimiter='|')
        for row in reader:
            rowValues = list(row.values())
            risk = priest(*rowValues[1:])
            rowValues.append(risk)
            hospital = find_hospital(row['age'], row['sex'], risk)
            rowValues.append(hospital)
            address = get_address(hospital)
            rowValues.append(address)
            processedPeople[row['patient']] = rowValues[1:]

    return processedPeople


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [ ]:
processedPeople = process_people('/data/people.psv')

import pandas as pd
import json

with open('/data/people_results.json') as jsonFile:
    peopleResults = json.load(jsonFile)
    
    for key in data:
        peopleResults[key][1] = int(peopleResults[key][1])
        peopleResults[key][2] = int(peopleResults[key][2])
        peopleResults[key][4] = int(peopleResults[key][4])
        peopleResults[key][5] = int(peopleResults[key][5])
        processedPeople[key][1] = int(processedPeople[key][1])
        processedPeople[key][2] = int(processedPeople[key][2])
        processedPeople[key][3] = float(processedPeople[key][3])
        processedPeople[key][4] = int(processedPeople[key][4])
        processedPeople[key][5] = int(processedPeople[key][5])
        processedPeople[key][6] = float(processedPeople[key][6])

peopleResultsDf = pd.DataFrame(peopleResults.items())
processedPeopleDf = pd.DataFrame(processedPeople.items())

assert peopleResultsDf.equals(processedPeopleDf)

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')